In [1]:
from typesense_retrieval import TypesenseRetrieval
import typesense

qa = TypesenseRetrieval()

# Initialize Typesense client
client = typesense.Client({
    'nodes': [{
        'host': 'localhost',
        'port': '8108',
        'protocol': 'http'
    }],
    'api_key': 'xyz',
    'connection_timeout_seconds': 60,
    'retry_interval_seconds': 5
})

In [12]:
question = "Tell me about the nursing program"

search_parameters = {
  'q'         : question,
  'query_by'  : 'embedding,context',
  'prefix'    : False,
  'limit'     : 3,
  #'filter_by' : 'url:https://www2.brockport.edu/academics/computing-sciences/'
}

response = client.collections['brockport_data_v1'].documents.search(search_parameters)

results = []
for response in response['hits']:
    results.append({
        'context': response['document']['context'], 
        'url': response['document']['url'], 
        'hybrid_search_info': response['hybrid_search_info'],
        'text_match_info': response['text_match_info']
    })
results

[{'context': 'This was my first experience to see anything of historical importance from those events, and I think being able to actually see it puts it into perspective. History is incredibly important to me, and I am a firm believer that if we don’t learn about it and contextualize it, we’ll repeat it.',
  'url': 'https://www2.brockport.edu/academics/honors-college/newsletter/',
  'hybrid_search_info': {'rank_fusion_score': 0.699999988079071},
  'text_match_info': {'best_field_score': '2216159019008',
   'best_field_weight': 15,
   'fields_matched': 1,
   'score': '1733920950581198969',
   'tokens_matched': 3}},
 {'context': 'The graduate program builds on the fundamental principles of nursing to ensure the provision of high quality care. Students learn critical assessment, diagnosis, and treatment skills and are mentored in the application of these skills throughout their clinical rotations.',
  'url': 'https://www2.brockport.edu/academics/nursing/family-nurse-practitioner/',
  'hyb

In [13]:
search_urls = qa._get_text_retrieval_places(question)
search_urls

{'main_categories': ['academics'], 'subcategories': ['academics-nursing']}

In [14]:
main_url = search_urls['main_categories'][0]

urls = []
base = "https://www2.brockport.edu/"
for url in search_urls['main_categories']:
    urls.append(base + url)

for url in search_urls['subcategories']:
    urls.append(base + main_url + '/' + (url.split('-', 1)[1] if '-' in url else url))

In [15]:
response = client.multi_search.perform(
    search_queries={
        'searches': [{
                'q': question,
                'limit': 3,
                'filter_by': f"url:{str(urls)}"
            }]
        },
    common_params = {
        'collection': 'brockport_data_v1',
        'query_by': 'embedding,context',
        'prefix': False,
    }
)

In [16]:
results = []
for response in response['results'][0]['hits']:
    results.append({
        'context': response['document']['context'], 
        'url': response['document']['url'], 
        'hybrid_search_info': response['hybrid_search_info'],
        'text_match_info': response['text_match_info']
    })
results

[{'context': 'Our 100% online Doctorate of Nursing Program is the first doctorate level nursing program in the SUNY comprehensive sector and serves as the most advanced degree in the field of nursing.',
  'url': 'https://www2.brockport.edu/academics/nursing/',
  'hybrid_search_info': {'rank_fusion_score': 1.0},
  'text_match_info': {'best_field_score': '517734',
   'best_field_weight': 102,
   'fields_matched': 3,
   'score': '1060320051',
   'tokens_matched': 0}},
 {'context': 'The Department of Nursing will be conducting an accreditation visit through the Commission on Collegiate Nursing Education (CCNE) on February 28, 29, and March 1, 2024. CCNE provides the opportunity for program constituents and other interested parties to submit, in writing, comments concerning a program’s qualifications for accreditation.',
  'url': 'https://www2.brockport.edu/academics/nursing/',
  'hybrid_search_info': {'rank_fusion_score': 0.42500001192092896},
  'text_match_info': {'best_field_score': '513